## 17/12/2020 Update
# Linking and Comparing Local Connect Stats with Low-rank Recurrent Network
* From Local Connect Stats cite[Sharpee]
* From Low-rank structure to EI 

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
from sympy import *
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

## Check the results in Sharpee's paper
* local stats, EE,EI,IE,II occupy their own properties, independently
* no pair correlation


In [178]:
Nt=np.array([2000,1000])
NE,NI=Nt[0],Nt[1]
at = Nt/N
N=np.sum(Nt)
aee,aei,aie,aii=4.5,-6.0,5.0,-2.5
JEEsub,JEIsub,JIEsub,JIIsub=aee*np.ones((NE,NE))/NE,aei*np.ones((NE,NI))/NI,aie*np.ones((NI,NE))/NE,aii*np.ones((NI,NI))/NI
J = np.zeros((NE+NI,NE+NI))

# determine gcd
Gmat=np.zeros((2,2))
Gmatorg=np.array([[0.8,1.2],[0.9,0.2]])
# Gmat/=np.sqrt(N)
for i in range(2):
    for j in range(2):
        Gmat[i,j]=Gmatorg[i,j]/np.sqrt(N)
# EE
gij=Gmat[0,0]*np.random.randn(NE,NE)
J[:NE,:NE]=gij.copy()
# EI
gij=Gmat[0,1]*np.random.randn(NE,NI)
J[:NE,NE:]=gij.copy()
# IE
gij=Gmat[1,0]*np.random.randn(NI,NE)
J[NE:,:NE]=gij.copy()
# II
gij=Gmat[1,1]*np.random.randn(NI,NI)
J[NE:,NE:]=gij.copy()

P = np.zeros((N,N))
P[:NE,:NE],P[:NE,NE:],P[NE:,:NE],P[NE:,NE:]=JEEsub.copy(),JEIsub.copy(),JIEsub.copy(),JIIsub.copy()
X = P.copy()+J.copy()

In [113]:
gavg=np.sqrt(np.sum(np.sum(Gmat**2)))/2.0
print('averaging g noise:',gavg*np.sqrt(N))

averaging g noise: 0.8558621384311844


In [105]:
aee,aei,aie,aii=4.5,-6.0,5.0,-2.5
Asmall=np.array([[aee,aei],[aie,aii]])
eigva,_=la.eig(Asmall)
_,sigva,_=la.svd(Asmall)
print('eigenvalue:',eigva,'singular value:',sigva,'eigG',np.sqrt(eigG[0]))

eigenvalue: [1.+4.21307489j 1.-4.21307489j] singular value: [9.12570385 2.05463604] eigG 0.8546614831429354


In [47]:
# calculate singular value circles
_,sigvrand,_=la.svd(J)
fig,ax=plt.subplots(figsize=(6,6))
n, bins, patches = ax.hist(sigvrand, 100, density=True, alpha=0.5)
# ax.scatter(np.real(eigvrand),np.imag(eigvrand),s=5.0,c='blue',alpha=0.5)

In [179]:
# all about calculate eigenvalues
eigvrand,eigvecrand=la.eig(J)
eigstruct,eigvecstruct=la.eig(P)
# summation
X = P.copy()+J.copy()
eigvx,eigvecx=la.eig(X)
# theoretical
M = np.zeros((2,2))
for i in range(2):
    for j in range(2):
        M[i,j]=(Gmat[i,j]*np.sqrt(N))**2*(Nt[j]/N)
eigG,eigvecG=la.eig(M)
# low rank small
Asmall=np.array([[aee,aei],[aie,aii]])
eigva,_=la.eig(Asmall)


In [180]:
ticks = np.linspace(-2.0,2.0,3)
lims= [-2.1,2.1]

fig,ax=plt.subplots(figsize=(6,6))
ax.plot(np.sqrt(eigG[0])*np.ones(100),np.linspace(-1.0,1.0,100))
ax.scatter(np.real(eigvrand),np.imag(eigvrand),s=5.0,c='blue',alpha=0.5)
ax.scatter(np.real(eigstruct[:4]),np.zeros(4),s=15.0,c='red',alpha=0.5)
# ax.set_xlim(lims)
# ax.set_ylim(lims)
# ax.set_xticks(ticks)
# ax.set_yticks(ticks)
ax.set_aspect('equal')
ax.grid()

## relate this to low rank (here rank-2)

In [109]:
lvecX,svX,rvecXh=la.svd(X)
rvecX=rvecXh.T
m=lvecX[:,:2].copy()
n=rvecX[:,:2].copy()
n[:,0]*=svX[0]
n[:,1]*=svX[1]

# n[:,1]=rvecX[:,2]*svX[2]
# m[:,1]=lvecX[:,2]

In [110]:
print('singular value:',svX[:4])

singular value: [9.21533954 2.36599547 1.84315887 1.83814427]


In [111]:
morg,norg=m,n
fig,ax=plt.subplots(4,4,figsize=(9,9))
for i in range(2):
    for j in range(2):
        ax[i][j].scatter(morg[:NE,i],morg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[i][j].scatter(morg[NE:,i],morg[NE:,j],s=5.0,c='blue',alpha=0.05)
for i in range(2):
    for j in range(2):
        ax[2+i][2+j].scatter(norg[:NE,i],norg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[2+i][2+j].scatter(norg[NE:,i],norg[NE:,j],s=5.0,c='blue',alpha=0.05)
        
for i in range(2):
    for j in range(2):
        ax[i][2+j].scatter(morg[:NE,i],norg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[i][2+j].scatter(morg[NE:,i],norg[NE:,j],s=5.0,c='blue',alpha=0.05)
for i in range(2):
    for j in range(2):
        ax[2+i][j].scatter(norg[:NE,i],morg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[2+i][j].scatter(norg[NE:,i],morg[NE:,j],s=5.0,c='blue',alpha=0.05)
for i in range(4):
    for j in range(4):
        # ax[i][j].set_xlim(lims)
        # ax[i][j].set_ylim(lims)
        # ax[i][j].set_xticks(ticks)
        # ax[i][j].set_yticks(ticks)
        ax[i][j].set_aspect('equal')
        ax[i][j].grid()

In [117]:
# test Dale's law
Nt=np.array([300,100])
NE,NI=Nt[0],Nt[1]
N=np.sum(Nt)
aee,aei,aie,aii=4.5,-6.0,5.0,-2.5
JEEsub,JEIsub,JIEsub,JIIsub=aee*np.ones((NE,NE))/NE,aei*np.ones((NE,NI))/np.sqrt(NE*NI),aie*np.ones((NI,NE))/np.sqrt(NE*NI),aii*np.ones((NI,NI))/NI
P = np.zeros((N,N))
P[:NE,:NE],P[:NE,NE:],P[NE:,:NE],P[NE:,NE:]=JEEsub.copy(),JEIsub.copy(),JIEsub.copy(),JIIsub.copy()
eigvP,_=la.eig(P)
_,sigvp,_=la.svd(P)
Asmall=np.array([[aee,aei],[aie,aii]])
eigva,_=la.eig(Asmall)
_,sigva,_=la.svd(Asmall)

print('eigenvalue:',eigvP[:2],eigva[:2])
print('singular value:',sigvp[:2],sigva[:2])

eigenvalue: [1.+4.21307489j 1.-4.21307489j] [1.+4.21307489j 1.-4.21307489j]
singular value: [9.12570385 2.05463604] [9.12570385 2.05463604]


In [184]:
# test Dale's law
Nt=np.array([100,1000])
NE,NI=Nt[0],Nt[1]
N=np.sum(Nt)
at=Nt/N
# determine gcd
Gmat=np.zeros((2,2))
Gmatorg=np.array([[0.8,1.2],[0.9,0.2]])
# Gmat/=np.sqrt(N)
for i in range(2):
    for j in range(2):
        Gmat[i,j]=Gmatorg[i,j]/np.sqrt(N)
# EE
J=np.zeros((N,N))

gij=np.zeros((NE,NE))
gij=Gmat[0,0]*np.random.randn(NE,NE)
J[:NE,:NE]=gij.copy()
# EI
gij=np.zeros((NE,NI))
gij=Gmat[0,1]*np.random.randn(NE,NI)
J[:NE,NE:]=gij.copy()
# IE
gij=np.zeros((NI,NE))
gij=Gmat[1,0]*np.random.randn(NI,NE)
J[NE:,:NE]=gij.copy()
# II
gij=np.zeros((NI,NI))
gij=Gmat[1,1]*np.random.randn(NI,NI)
J[NE:,NE:]=gij.copy()

gavg=0
for i in range(2):
    for j in range(2):
        gavg+=np.sqrt(Gmatorg[i,j]**2*at[i]*at[j])
M = np.zeros((2,2))
for i in range(2):
    for j in range(2):
        M[i,j]=(Gmatorg[i,j]**2)*at[j]

eigvj,_=la.eig(J)
eigvM,_=la.eig(M)
ticks = np.linspace(-2.0,2.0,3)
lims= [-2.1,2.1]

fig,ax=plt.subplots(figsize=(6,6))
ax.plot(np.sqrt(eigvM[0])*np.ones(100),np.linspace(-1.0,1.0,100))
ax.scatter(np.real(eigvj),np.imag(eigvj),s=5.0,c='blue',alpha=0.5)
print('average eigv:',gavg)


average eigv: 0.858253007850327


In [227]:
# testing distribution of J_{ij}
NE,NI=750,250
sigm,sign=0.5,1.2
mE,nE=sigm*np.random.randn(NE,1)/np.sqrt(NE),sign*np.random.randn(NE,1)/np.sqrt(NE)
JEE = mE@nE.T 
eigv,eigvec=la.eig(JEE)
fig,ax=plt.subplots(figsize=(6,6))
ax.scatter(np.real(eigvj),np.imag(eigvj),s=5.0,c='blue',alpha=0.5)

JEEoff=JEE.copy()
JEEdiag=np.zeros(NE)
for i in range(NE):
    JEEoff[i,i:NE-1]=JEEoff[i,i+1:]
    JEEdiag[i]=JEE[i,i]
fig,ax=plt.subplots(2,1,figsize=(6,6))
Joff=JEEoff[:,:NE-1]
Joff=np.reshape(Joff,(NE**2-NE,1))
n, bins, patches = ax[0].hist(Joff, 100, density=True, alpha=0.5)
n, bins, patches = ax[1].hist(JEEdiag, 100, density=True, alpha=0.5)
print(sigm*sign)



0.6


## What's the distribution of $J_{i,j}$ in $\tilde{\mathbf{X}}=\mathbf{X}+\mathbf{p}$

In [229]:
# testing distribution of J_{ij} in \tilde{X}
NE,NI=2000,250
sigm,sign,grandom=0.6,1.0,0.5
rho=0.0
zetax,zetay=np.random.randn(NE,1),np.random.randn(NE,1)
m = sigm*zetax/np.sqrt(NE)
n =sign*(rho*zetax+np.sqrt(1-rho**2)*zetay)/np.sqrt(NE)
P = m@n.T 
X = (grandom)*np.random.randn(NE,NE)/np.sqrt(NE)

Xtilde=X+P

eigv,eigvec=la.eig(Xtilde)
fig,ax=plt.subplots(figsize=(6,6))
ax.scatter(np.real(eigv),np.imag(eigv),s=5.0,c='blue',alpha=0.5)

JEEoff=Xtilde.copy()
JEEdiag=np.zeros(NE)
for i in range(NE):
    JEEoff[i,i:NE-1]=JEEoff[i,i+1:]
    JEEdiag[i]=Xtilde[i,i]
fig,ax=plt.subplots(2,1,figsize=(6,6))
Joff=JEEoff[:,:NE-1]
Joff=np.reshape(Joff,(NE**2-NE,1))
n, bins, patches = ax[0].hist(Joff, 100, density=True, alpha=0.5)
n, bins, patches = ax[1].hist(JEEdiag, 100, density=True, alpha=0.5)

## Generate Random Block Matrix, embedding low rank structur $\tilde{\mathbf{X}}_{\mathrm{type}}=\mathbf{X}_{\mathrm{type}}+\mathbf{P}_{\mathrm{type}}$

In [292]:
Nt=np.array([2000,2000])
NE,NI=Nt[0],Nt[1]
N=NE+NI
at = Nt/N
N=np.sum(Nt)
xee,xei,xie,xii=5.5,2.5,6.0,1.25
# subject to
sigmE,signI,sigmI,signE=2.0,3.0,1.0,4.0
rhoE=xee/sigmE/signE*1.5
rhoI=xii/sigmI/signI*1.5
X,P=np.random.randn(N,N),np.zeros((N,N))
X[:NE,:NE],X[:NE,NE:]=xee*X[:NE,:NE]/np.sqrt(N),xei*X[:NE,NE:]/np.sqrt(N)
X[NE:,:NE],X[NE:,NE:]=xie*X[NE:,:NE]/np.sqrt(N),xii*X[NE:,NE:]/np.sqrt(N)
m,n=np.zeros((N,1)),np.zeros((N,1))
zetaxE,zetayE,zetaxI,zetayI=np.random.randn(NE),np.random.randn(NE),np.random.randn(NI),np.random.randn(NI)
m[:NE,0]=sigmE*zetaxE/np.sqrt(N)
n[:NE,0]=signE*(rhoE*zetaxE+np.sqrt(1-rhoE**2)*zetayE)/np.sqrt(N)
m[NE:,0]=sigmI*zetaxI/np.sqrt(N)
n[NE:,0]=signI*(rhoI*zetaxI+np.sqrt(1-rhoI**2)*zetayI)/np.sqrt(N)
P=m@n.T 
Xtilde=X.copy()+P.copy()



In [238]:
phiacttt=np.zeros((N,1))
phiacttt[:NE,0],phiacttt[NE:,0]=(rhoE*zetaxE+np.sqrt(1-rhoE**2)*zetayE)/np.sqrt(N),(rhoI*zetaxI+np.sqrt(1-rhoI**2)*zetayI)/np.sqrt(N)
eitatt=Xtilde@phiacttt
eitarand=X@phiacttt
coveitatt=eitatt@eitatt.T 
coveitarand=eitarand@eitarand.T 
deltaEt,deltaEr=coveitatt[:NE,:NE],coveitarand[:NE,:NE]
deltaIt,deltaIr=coveitatt[NE:,NE:],coveitarand[NE:,NE:]
print('std, Et:',np.std(deltaEt),'; random:',np.std(deltaEr))
print('std, It:',np.std(deltaIt),'; random:',np.std(deltaIr))

In [411]:
eigvx,_=la.eig(X)
eigvxt,_=la.eig(Xtilde)
fig,ax=plt.subplots(2,2,figsize=(9,9))
ax[0][0].scatter(np.real(eigvx),np.imag(eigvx),s=2.0,c='red',alpha=0.5)
ax[0][0].set_aspect('equal')
print(np.sqrt(xee**2*NE/N))
ax[1][0].scatter(np.real(eigvxt),np.imag(eigvxt),s=2.0,c='red',alpha=0.5)
ax[1][0].set_aspect('equal')
# HISTOGRAM
n, bins, patches = ax[0][1].hist(np.real(eigvx), 20, density=True, alpha=0.5)
ax[0][1].set_xlim([-4.0,4.0])
n, bins, patches = ax[1][1].hist(np.real(eigvxt), 20, density=True, alpha=0.5)
ax[0][1].set_xlim([-4.0,4.0])
# SUBTYPE
XEI,XTEI=X[:NE,NE:],Xtilde[:NE,NE:]
eigvEI,eigvec=la.eig(XEI)
eigvTEI,eigvec=la.eig(XTEI)
fig,ax=plt.subplots(1,2,figsize=(6,6))
ax[0].scatter(np.real(eigvEI),np.imag(eigvEI),s=5.0,c='blue',alpha=0.5)
ax[1].scatter(np.real(eigvTEI),np.imag(eigvTEI),s=5.0,c='blue',alpha=0.5)

XEE,XTEE=X[:NE,:NE],Xtilde[:NE,:NE]
eigvEE,eigvec=la.eig(XEE)
eigvTEE,eigvec=la.eig(XTEE)
fig,ax=plt.subplots(1,2,figsize=(6,6))
ax[0].scatter(np.real(eigvEE),np.imag(eigvEE),s=5.0,c='red',alpha=0.5)
ax[1].scatter(np.real(eigvTEE),np.imag(eigvTEE),s=5.0,c='red',alpha=0.5)

XIE,XTIE=X[NE:,:NE],Xtilde[NE:,:NE]
eigvIE,eigvec=la.eig(XIE)
eigvTIE,eigvec=la.eig(XTIE)
fig,ax=plt.subplots(1,2,figsize=(6,6))
ax[0].scatter(np.real(eigvIE),np.imag(eigvIE),s=5.0,c='blue',alpha=0.5)
ax[1].scatter(np.real(eigvTIE),np.imag(eigvTIE),s=5.0,c='blue',alpha=0.5)

XII,XTII=X[NE:,NE:],Xtilde[NE:,NE:]
eigvII,eigvec=la.eig(XII)
eigvTII,eigvec=la.eig(XTII)
fig,ax=plt.subplots(1,2,figsize=(6,6))
ax[0].scatter(np.real(eigvII),np.imag(eigvII),s=5.0,c='red',alpha=0.5)
ax[1].scatter(np.real(eigvTII),np.imag(eigvTII),s=5.0,c='red',alpha=0.5)


1.7677669529663689


## reduced structure

In [293]:
gmat = np.array([[xee,xei],[xie,xii]])
gmat/=2.0
eigvgmat,_=la.eig(gmat)
gaverage=np.sqrt(xee**2+xei**2+xie**2+xii**2)/2.0
print('small g:',eigvgmat[:],gaverage)
# _,svgmat,_=la.svd(gmat)
# print('svd gmat:',2*svgmat*np.sqrt(2))



small g: [ 3.89632463 -0.52132463] 4.302978619514627


## adding structure in random part

In [313]:
ugm,svgm,vhgm=la.svd(gmat)
print('gmat singular:',svgm[:],ugm)
eigvgm,eigvecgm=la.eig(gmat)
print('gmat eigenvalue:',eigvgm[:])
ngm=vhgm.copy()
ngm=ngm.T 
ngm[:,0],ngm[:,1]=svgm[0]*ngm[:,0],svgm[1]*ngm[:,1]
JE,JI=2.0*1.6,1.8*2.0
Am=np.zeros((N,N))
Am[:,:NE],Am[:,NE:]=JE/NE,-JI/NI
ua,svam,vha=la.svd(Am)
va=vha.T
# ua[:,0]=-1.0/np.sqrt(N)
# va[:NE,0]=JE/np.sqrt(NE)
# va[NE:,0]=-JI/np.sqrt(NI)
# va[:,0]/=-np.linalg.norm(va[:,0])




gmat singular: [4.27668552 0.47495894] [[-0.70188878 -0.71228656]
 [-0.71228656  0.70188878]]
gmat eigenvalue: [ 3.89632463 -0.52132463]


In [315]:
Xtotal=X.copy()+Am.copy()
# singular decomposition
uxt,svxt,vhxt=la.svd(Xtotal)
vxt=vhxt.T
print(svxt[:5])
ux,svx,vhx=la.svd(X)
vx=vhx.T
print(svx[:5])


[10.11458589  9.92046626  9.87057854  9.85892891  9.84010743]
[9.92602229 9.88290686 9.86440262 9.8458118  9.82897575]


In [316]:
muxtt=np.zeros((2,1))
muxtt[0,0],muxtt[1,0]=np.mean(uxt[:NE,0]),np.mean(uxt[NE:,0])
mua = np.zeros((2,1))
mua[0,0],mua[1,0]=np.mean(ua[:NE,0]),np.mean(ua[NE:,0])
mvxtt=np.zeros((2,1))
mvxtt[0,0],mvxtt[1,0]=np.mean(vxt[:NE,0]),np.mean(vxt[NE:,0])
mva = np.zeros((2,1))
mva[0,0],mva[1,0]=np.mean(va[:NE,0]),np.mean(va[NE:,0])

In [317]:
fig,ax=plt.subplots(2,2,figsize=(9,9))
ax[1][1].scatter(uxt[:NE,0],uxt[NE:,0],s=10.0,c='red',alpha=0.5)
ax[1][1].scatter(vxt[:NE,0],vxt[NE:,0],s=10.0,c='blue',alpha=0.5)
ax[1][1].scatter(ua[NE-3,0],ua[NE+3,0],s=25.0,c='green',alpha=1.0)
ax[1][1].scatter(va[NE-2,0],va[NE+2,0],s=25.0,c='green',alpha=1.0)
ax[1][1].scatter(muxtt[0,0],muxtt[1,0],s=25.0,c='black',alpha=1.0)
ax[1][1].scatter(mvxtt[0,0],mvxtt[1,0],s=25.0,c='black',alpha=1.0)
vector_1= np.squeeze(muxtt)
vector_2 = np.squeeze(mua)
angleu=calculate_angle(vector_1,vector_2)
vector_1= np.squeeze(mvxtt)
vector_2 = np.squeeze(mva)
anglev=calculate_angle(vector_1,vector_2)
print('angle u and v:',angleu,anglev)
vector_1= np.squeeze(mva)
vector_2 = np.squeeze(mua)
anglecrossorg=calculate_angle(vector_1,vector_2)
vector_1= np.squeeze(mvxtt)
vector_2 = np.squeeze(muxtt)
anglecrossaft=calculate_angle(vector_1,vector_2)
print('angle u and v:',anglecrossorg,anglecrossaft)


angle u and v: 0.02037143527636012 0.28445116211170546
angle u and v: 1.6295521495106249 1.3654724226752795


In [329]:
print('ratio ju:',mvxtt/mva,svxt[0])
print('sinvec:',np.sum(uxt[:NE,0]*vxt[:NE,0])/np.sum(ua[:NE,0]*va[:NE,0]))
print('sinvec:',np.sum(uxt[NE:,0]*vxt[NE:,0])/np.sum(ua[NE:,0]*va[NE:,0]))

print('sinvec:',np.std(vxt[:NE,0])/np.std(vxt[NE:,0]))
print('sinvec:',np.std(uxt[:NE,0])/np.std(uxt[NE:,0]))

# print('sinvec:',np.sum(va[:NE,0]*va[:NE,0])/np.sum(va[NE:,0]*vxt[NE:,0]))

ratio ju: [[0.76078738]
 [0.42369813]] 10.11458588657317
sinvec: 0.4394546146874324
sinvec: 0.25464103074642996
sinvec: 5.601988919461165
sinvec: 0.8578186579626327


In [319]:
# fig,ax=plt.subplots(2,2,figsize=(9,9))
ax[1][0].scatter(uxt[:NE,0],vxt[:NE,0],s=5.0,c='red',alpha=0.05)
ax[1][0].scatter(uxt[NE:,0],vxt[NE:,0],s=5.0,c='blue',alpha=0.05)
ax[1][0].scatter(ua[NE-3,0],va[NE-3,0],s=25.0,c='black',alpha=1.0)
ax[1][0].scatter(ua[NE+2,0],va[NE+2,0],s=25.0,c='black',alpha=1.0)
ax[1][0].scatter(muxtt[0,0],mvxtt[0,0],s=25.0,c='green',alpha=1.0)
ax[1][0].scatter(muxtt[1,0],mvxtt[1,0],s=25.0,c='green',alpha=1.0)

In [276]:
vt1=mva.copy()
vt2=ugm.T@vt1
anglet =calculate_angle(np.squeeze(vt2),np.squeeze(vt1))
print(anglet)

1.5095303432414557


In [299]:
def calculate_angle(vector_1,vector_2):
    unit_vector_1 = vector_1 /np.linalg.norm(vector_1)
    unit_vector_2 = vector_2 /np.linalg.norm(vector_2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)
    return angle

# vector_1= muxtt
# vector_2 = mua
# unit_vector_1 = vector_1 / np. linalg. norm(vector_1)
# unit_vector_2 = vector_2 / np. linalg. norm(vector_2)
# dot_product = np. dot(unit_vector_1, unit_vector_2)
# angle = np. arccos(dot_product)

In [150]:

# coefficient
muxtt=np.zeros((2,1))
muxtt[0,0],muxtt[1,0]=np.mean(uxt[:NE,0]),np.mean(uxt[NE:,0])
mua = np.zeros((2,1))
mua[0,0],mua[1,0]=np.mean(ua[:NE,0]),np.mean(ua[NE:,0])
print(mua,muxtt)

alphau2=muxtt@mua.T@la.inv(mua@mua.T)
alphauE=np.sum((uxt[:NE,0]-muxtt[0,0])*(uxt[:NE,0]-muxtt[0,0]))**2
alphauI=np.sum((uxt[NE:,0]-muxtt[1,0])*(uxt[NE:,0]-muxtt[1,0]))**2
alphaucross=np.sum((uxt[:NE,0]-muxtt[0,0])*(uxt[NE:,0]-muxtt[1,0]))**2
alphauI=np.sum((uxt[NE:,0]-muxtt[1,0])*(uxt[NE:,0]-muxtt[1,0]))**2
# compare
print('beta:',alphauE,alphauI,alphaucross,'; params:',2*svgm[0]/svam[0])

[[-0.02236068]
 [-0.02236068]] [[-0.01822479]
 [-0.0226821 ]]
beta: 0.0011014403391343501 0.014445953274896447 1.022859548322422e-05 ; params: 0.7591500420761472


In [107]:
noise=np.array([[alphauE],[alphauI]])
invnoise=la.inv(ngm@ngm.T)@noise
print('org noise:',invnoise[0]/invnoise[1])
print('ua:',muxtt[0]/muxtt[1])
# square=gmat@np.ones((2,1))/np.sqrt(N)
# print(square[0]/square[1])
# ratiotest=np.sum(square**2,axis=0)
# print(ratiotest[0]/ratiotest[1])

org noise: [-1.40573017]
ua: [1.21802445]


In [149]:
# ua,sva,vha=la.svd(Am)
# coefficient
mvxtt=np.zeros((2,1))
mvxtt[0,0],mvxtt[1,0]=np.mean(vxt[:NE,0]),np.mean(vxt[NE:,0])
mva = np.zeros((2,1))
mva[0,0],mva[1,0]=np.mean(va[:NE,0]),np.mean(va[NE:,0])
print(mva,mvxtt)
alphavE=np.sum((vxt[:NE,0]-mvxtt[0,0])*(vxt[:NE,0]-mvxtt[0,0]))**2
alphavI=np.sum((vxt[NE:,0]-mvxtt[1,0])*(vxt[NE:,0]-mvxtt[1,0]))**2
alphavcross=np.sum((vxt[:NE,0]-mvxtt[0,0])*(vxt[NE:,0]-mvxtt[1,0]))**2
# compare
print('beta:',alphavE,alphavI,alphaucross)

[[-0.01071125]
 [ 0.02975347]] [[-0.0107973 ]
 [ 0.02479937]]
beta: 0.06824677342412665 5.138802295543543e-05 1.022859548322422e-05


In [147]:
macos=(mva.T@mua)
macos/=np.sqrt(np.sum(mva**2))
macos/=np.sqrt(np.sum(mua**2))
mxttcos=(mvxtt.T@muxtt)
mxttcos/=np.sqrt(np.sum(mvxtt**2))
mxttcos/=np.sqrt(np.sum(muxtt**2))
print(macos,mxttcos)

[[-0.42579704]] [[-0.46470134]]


In [148]:
muxtt_=ngm@mua
# mvxtt_=ngm@mva
macos=(mva.T@mua)
macos/=np.sqrt(np.sum(mva**2))
macos/=np.sqrt(np.sum(mua**2))
mxttcos=(mvxtt.T@muxtt_)
mxttcos/=np.sqrt(np.sum(mvxtt**2))
mxttcos/=np.sqrt(np.sum(muxtt_**2))
print(macos,mxttcos)

[[-0.42579704]] [[-0.01586666]]


# back up code

In [333]:
fig,ax=plt.subplots(figsize=(9,9))
ax.plot(svx[:])
print(np.max(svx))
overlap=np.zeros(2000)
for idx in range(2000):
    overlap[idx]=np.sum(lxvec[:,idx]*rxvec[:,idx])*svx[idx]
n, bins, patches = ax.hist(overlap, nbins, density=True, alpha=0.5)
print(np.max(overlap))


7.459298334656558
0.26243786874485775


In [284]:
stdme=np.std(lxvec[:NE,0])
stdmi=np.std(lxvec[NE:,0])
stdne=np.std(svx[0]*rxvec[:NE,0])
stdni=np.std(svx[0]*rxvec[NE:,0])
rm1=np.array([stdme,stdmi])
rm1=np.reshape(rm1,(2,1))
rn1=np.array([stdne,stdni])
rn1=np.reshape(rn1,(2,1))
stdme=np.std(lxvec[:NE,1])
stdmi=np.std(lxvec[NE:,1])
stdne=np.std(svx[1]*rxvec[:NE,1])
stdni=np.std(svx[1]*rxvec[NE:,1])
rm2=np.array([stdme,stdmi])
rm2=np.reshape(rm2,(2,1))
rn2=np.array([stdne,stdni])
rn2=np.reshape(rn2,(2,1))
gstdest=rm1@rn1.T +rm2@rn2.T 
print('cal g:',gstdest)
print('org g:',xee**2/N,xei**2/N,xie**2/N,xii**2/N)

cal g: [[0.00089115 0.00017327]
 [0.00276879 0.00053832]]
org g: 0.000625 0.0009 0.0025 0.000225


In [269]:
lxvec,svx,rxvech=la.svd(P)
rxvec=rxvech.T 
fig,ax=plt.subplots(2,2,figsize=(9,9))
nbins=30
n, bins, patches = ax[0][0].hist(lxvec[:NE,0], nbins, density=True, alpha=0.5)
n, bins, patches = ax[1][0].hist(lxvec[NE:,0], nbins, density=True, alpha=0.5)
n, bins, patches = ax[0][1].hist(rxvec[:NE,0], nbins, density=True, alpha=0.5)
n, bins, patches = ax[1][1].hist(rxvec[NE:,0], nbins, density=True, alpha=0.5)
